In [1]:
import pandas as pd
import numpy as np

In [2]:
min_data_value = "2000-01-01"
max_data_value = "2020-01-01"

In [3]:
symbols = pd.read_csv('data/symbols_valid_meta.csv')
symbols = symbols[symbols['ETF'] == 'N'] # keep only stocks 
symbols.head()

,Nasdaq Traded,Symbol,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
0,Y,A,"Agilent Technologies, Inc. Common Stock",N,,N,100.0,N,NaN,A,A,N
1,Y,AA,Alcoa Corporation Common Stock,N,,N,100.0,N,NaN,AA,AA,N
3,Y,AACG,ATA Creativity Global - American Depositary Sh...,Q,G,N,100.0,N,N,NaN,AACG,N
5,Y,AAL,"American Airlines Group, Inc. - Common Stock",Q,Q,N,100.0,N,N,NaN,AAL,N
6,Y,AAMC,Altisource Asset Management Corp Com,A,,N,100.0,N,NaN,AAMC,AAMC,N


In [4]:
print(f'Number of stocks: {symbols.count()["Symbol"]}')

Number of stocks: 5884


In [5]:
# Focus on 'G' segment
g_segment = symbols[symbols['Market Category'] == 'G']
print(f'Number of stocks in G segment: {g_segment.count()["Symbol"]}')

Number of stocks in G segment: 505


In [6]:
# Check historical data availability and reduce stock list

g_segment_selected = []

for symbol in g_segment['Symbol']:
    hist_data_tmp = pd.read_csv(f'data/stocks/{symbol}.csv')
    min_date = hist_data_tmp['Date'].min()
    max_date = hist_data_tmp['Date'].max()
    if min_date <= min_data_value and max_date >= max_data_value:
        g_segment_selected.append(symbol)

print(f'Number of stocks in G segment with sufficient historical data: {len(g_segment_selected)}')

Number of stocks in G segment with sufficient historical data: 129


In [7]:
volume_list = {}

# Get volume between 1 janv 2000 and 1 janv 2002
# Calculate average volume
# Compare to median volume and keep those above median

min_data_value_volume = min_data_value
max_date_value_volume = "2002-01-01"

for symbol in g_segment_selected:
    hist_data_tmp = pd.read_csv(f'data/stocks/{symbol}.csv')
    hist_data_tmp = hist_data_tmp[(hist_data_tmp['Date'] >= min_data_value_volume) & (hist_data_tmp['Date'] <= max_date_value_volume)]
    avg_volume = hist_data_tmp['Volume'].mean()
    volume_list[symbol] = avg_volume

volume_above_median = []
median_volume = np.median(list(volume_list.values()))

for symbol, avg_volume in volume_list.items():
    if avg_volume > median_volume:
        volume_above_median.append(symbol)

volume_above_median = sorted(volume_above_median)

print(f'Number of stocks in G segment with volume above median: {len(volume_above_median)}')

Number of stocks in G segment with volume above median: 64


In [11]:
initial_data = pd.DataFrame()

for symbol in volume_above_median:
    hist_data_tmp = pd.read_csv(f'data/stocks/{symbol}.csv')
    hist_data_tmp = hist_data_tmp[(hist_data_tmp['Date'] >= min_data_value) & (hist_data_tmp['Date'] <= max_data_value)]
    hist_data_tmp = hist_data_tmp[['Date','Open','High','Low','Close','Adj Close','Volume']].copy()

    hist_data_tmp["High_over_Low"] = hist_data_tmp["High"] / hist_data_tmp["Low"]
    hist_data_tmp["Day_yield"] = np.log(hist_data_tmp["Close"] / hist_data_tmp["Open"])

    hist_data_tmp = hist_data_tmp.drop(columns=['High','Low','Close','Open'])
    
    # Rename columns to feature names
    hist_data_tmp.rename(columns={'Adj Close': 'Price', 'Volume': 'Volume',
                                  'High_over_Low': 'High_over_Low', 'Day_yield': 'Day_yield'}, inplace=True)

    # Set Date as index
    hist_data_tmp.set_index('Date', inplace=True)
    
    # Create MultiIndex columns: (symbol, feature)
    hist_data_tmp.columns = pd.MultiIndex.from_product([[symbol], hist_data_tmp.columns])
    
    # Forward fill
    hist_data_tmp.ffill(inplace=True)

    # Concatenate along columns (axis=1)
    if initial_data.empty:
        initial_data = hist_data_tmp
    else:
        initial_data = pd.concat([initial_data, hist_data_tmp], axis=1)

initial_data.reset_index(inplace=True)  # If you need Date as column later
initial_data.head()

Date    ALJJ                                        AMOT           \
                Price    Volume High_over_Low Day_yield     Price   Volume   
0  2000-01-03  29.000  234000.0      1.070175 -0.017094  1.643544  19600.0   
1  2000-01-04  27.625  180200.0      1.072917 -0.046417  1.643544  12100.0   
2  2000-01-05  27.250  182300.0      1.061611 -0.018182  1.526148   6000.0   
3  2000-01-06  26.750  119100.0      1.064286 -0.009302  1.760940   8800.0   
4  2000-01-07  26.000  257300.0      1.055556 -0.023754  1.760940   6500.0   

                               ASRV  ...          UNTY               WLFC  \
  High_over_Low Day_yield     Price  ... High_over_Low Day_yield    Price   
0      1.134615  0.000000  8.404096  ...      1.030303 -0.010050  6.62500   
1      1.092593  0.000000  8.491637  ...      1.030303  0.029853  6.37500   
2      1.076923 -0.074108  8.666723  ...      1.000000  0.000000  6.37500   
3      1.071429  0.068993  8.579179  ...      1.000000  0.000000  6.46875   
4      1.034483  0.000000  8.710496  ...      1.010417  0.010363  6.50000   

                                         WSTG                          \
     Volume High_over_Low Day_yield     Price    Volume High_over_Low   
0   17600.0      1.029126  0.014252  2.677632  130400.0      1.163462   
1    3500.0      1.039216 -0.038466  2.801597  209900.0      1.192308   
2  107300.0      1.030000  0.019803  2.826390  158500.0      1.070796   
3   32100.0      1.012165 -0.004819  2.628047   41100.0      1.110048   
4   31100.0      1.049020  0.009662  2.479289   42600.0      1.120000   

             
  Day_yield  
0 -0.105361  
1  0.036040  
2  0.008811  
3 -0.081493  
4 -0.058269  

[5 rows x 257 columns]

In [12]:
initial_data.to_csv('data/initial_data.csv', index=False)